update 

# Building a Quasi-Agent

A quasi-agent is like a lightweight version of an AI agent. it acts as a smart assistant that can follow instructions step by step,

but it can’t make decisions or learn from feedback like a full agent would.

In this case, we’re building one that:

- Asks the user what they need (like a chatbot),
- Generates Python code based on that input,
- Adds documentation (docstrings),
- Creates test cases using unittest.

It’s a controlled simulation of an AI assistant with memory, made just to practice managing context and prompt chaining.

------------------------------------------------------------------------------------------------------------

# Quasi-Agent Architecture:

The goal is to simulate an agent that helps write Python functions by walking through 3 distinct steps:

1- Initial code generation: {The quasi-agent asks the user for a function description.}

2- Documentation enhancement:  {1- The code from Step 1 is passed as input again. 

                              2- The agent adds a docstring to explain what the function does}

3- Test case creation: {1- The documented function is now used as input again.

                      2- The agent generates unit tests using Python’s unittest framework.}

In [2]:
import os
from dotenv import load_dotenv

# Get API key from environment variables
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

if api_key:
    print("API key loaded successfully!")
else:
    print("Error: API key not found in .env file")


API key loaded successfully!


In [8]:
from litellm import completion
from typing import List, Dict
import sys

def generate_response(messages: List[Dict]) -> str:
   """Call LLM to get response"""
   response = completion(
      model="openai/gpt-4o", # testing the performance of "4o" and "4o-mini" for saving-cost
      messages=messages,
      max_tokens=1000
   )
   return response.choices[0].message.content

def extract_code_block(response: str) -> str:
   """Extract code block from response"""

   if not '```' in response:
      return response

   code_block = response.split('```')[1].strip()
   # Check for "python" at the start and remove

   if code_block.startswith("python"):
      code_block = code_block[6:]

   return code_block

def develop_custom_function():
   # Get user input for function description
   print("\nWhat kind of function would you like to create?")
   print("Example: 'A function that calculates the factorial of a number'")
   print("Your description: ", end='')
   function_description = input().strip()

   # Initialize conversation with system prompt
   messages = [
      {"role": "system", "content": "You are a Python expert helping to develop a function."}
   ]

   # First prompt - Basic function
   messages.append({
      "role": "user",
      "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
   })
   initial_function = generate_response(messages)

   # Parse the response to get the function code
   initial_function = extract_code_block(initial_function)

   print("\n=== Initial Function ===")
   print(initial_function)

   # Add assistant's response to conversation
   messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})

   # Second prompt - Add documentation
   messages.append({
      "role": "user",
      "content": "Add comprehensive documentation to this function, including description, parameters, "
                 "return value, examples, and edge cases. Output the function in a ```python code block```."
   })
   documented_function = generate_response(messages)
   documented_function = extract_code_block(documented_function)
   print("\n=== Documented Function ===")
   print(documented_function)

   # Add documentation response to conversation
   messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})

   # Third prompt - Add test cases
   messages.append({
      "role": "user",
      "content": "Add unittest test cases for this function, including tests for basic functionality, "
                 "edge cases, error cases, and various input scenarios. Output the code in a \`\`\`python code block\`\`\`."
   })
   test_cases = generate_response(messages)
   # We will likely run into random problems here depending on if it outputs JUST the test cases or the
   # test cases AND the code. This is the type of issue we will learn to work through with agents in the course.
   test_cases = extract_code_block(test_cases)
   print("\n=== Test Cases ===")
   print(test_cases)

   # Generate filename from function description
   filename = function_description.lower()
   filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
   filename = filename.replace(' ', '_')[:30] + '.py'

   # Save final version
   with open(filename, 'w') as f:
      f.write(documented_function + '\n\n' + test_cases)

   return documented_function, test_cases, filename

if __name__ == "__main__":


   function_code, tests, filename = develop_custom_function()
   print(f"\nFinal code has been saved to {filename}")

<>:54: SyntaxWarning: invalid escape sequence '\`'
<>:54: SyntaxWarning: invalid escape sequence '\`'
<>:68: SyntaxWarning: invalid escape sequence '\`'
<>:68: SyntaxWarning: invalid escape sequence '\`'
<>:74: SyntaxWarning: invalid escape sequence '\`'
<>:54: SyntaxWarning: invalid escape sequence '\`'
<>:54: SyntaxWarning: invalid escape sequence '\`'
<>:68: SyntaxWarning: invalid escape sequence '\`'
<>:68: SyntaxWarning: invalid escape sequence '\`'
<>:74: SyntaxWarning: invalid escape sequence '\`'
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py:54: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py:54: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/var/folders/fd/sjwsgyf54kb5m8t3


What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: 
=== Initial Function ===

def sum_numbers(numbers):
    """
    This function takes a list of numbers and returns their sum.
    
    Parameters:
    numbers (list): A list of numeric values.
    
    Returns:
    float: The sum of the numbers in the list.
    """
    return sum(numbers)

=== Documented Function ===

def sum_numbers(numbers):
    """
    Calculate the sum of a list of numbers.

    This function takes a list of numeric values (integers or floats) 
    and returns their total sum. It can handle empty lists and 
    various numeric types.

    Parameters:
    numbers (list): A list of numeric values (int or float). 
                    The list can be empty, in which case the sum will be 0.

    Returns:
    float: The total sum of the numbers in the list. If the list is empty, 
           the function will return 0.0.

    Examples:
    >>>

/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py:54: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py:54: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py:68: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py:68: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_41125/3652606009.py

KeyboardInterrupt: 

In [7]:
from litellm import completion
from typing import List, Dict
import sys

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1000
    )
    return response.choices[0].message.content

def extract_email_content(response: str) -> str:
    """Extract email content from response"""
    if not '```' in response:
        return response
    
    # Get content between first set of triple backticks
    email_block = response.split('```')[1].strip()
    
    # Check if there's a language identifier and remove it
    if email_block.startswith("text"):
        email_block = email_block[4:].strip()
    
    return email_block

def create_email_assistant():
    # Get user input for email purpose
    print("\nWhat's the purpose of your email?")
    print("Example: 'I need to ask my manager for time off next week due to a family emergency.'")
    print("Your description: ", end='')
    email_purpose = input().strip()
    
    # Initialize conversation with system prompt
    messages = [
        {"role": "system", "content": "You are an expert email writing assistant."}
    ]
    
    # Initial Draft Generation
    messages.append({
        "role": "user", 
        "content": f"Write an email for the following purpose: {email_purpose}. Include an appropriate subject line. Output the email in a ```text code block```."
    })
    
    initial_draft = generate_response(messages)
    initial_draft = extract_email_content(initial_draft)
    print("\n=== Initial Draft ===")
    print(initial_draft)
    
    # Add assistant's response to conversation
    messages.append({"role": "assistant", "content": "```text\n\n" + initial_draft + "\n\n```"})
    
    # Tone & Grammar Refinement
    messages.append({
        "role": "user", 
        "content": "Enhance the professional tone, fix any grammar issues, and polish the language of this email. Output the improved email in a ```text code block```."
    })
    
    refined_email = generate_response(messages)
    refined_email = extract_email_content(refined_email)
    print("\n=== Refined Email ===")
    print(refined_email)
    
    # Add refined email to conversation
    messages.append({"role": "assistant", "content": "```text\n\n" + refined_email + "\n\n```"})
    
    # Generate Variants or Replies
    print("\nWhat would you like to do next?")
    print("1. Generate a more formal version")
    print("2. Generate a more casual version")
    print("3. Generate a shorter version")
    print("4. Create a follow-up email")
    print("Your choice (1-4): ", end='')
    
    choice = input().strip()
    
    if choice == '1':
        variant_prompt = "Create a more formal version of this email."
    elif choice == '2':
        variant_prompt = "Create a more casual version of this email."
    elif choice == '3':
        variant_prompt = "Create a shorter, more concise version of this email."
    elif choice == '4':
        print("\nWhat is the expected response from the recipient? ", end='')
        expected_response = input().strip()
        variant_prompt = f"Create a follow-up email assuming the recipient responded with: '{expected_response}'."
    else:
        print("Invalid choice. Defaulting to formal version.")
        variant_prompt = "Create a more formal version of this email."
    
    messages.append({
        "role": "user", 
        "content": f"{variant_prompt} Output the email in a ```text code block```."
    })
    
    variant_email = generate_response(messages)
    variant_email = extract_email_content(variant_email)
    print("\n=== Email Variant ===")
    print(variant_email)
    
    # save emails to files
    print("\nWould you like to save these emails to files? (y/n): ", end='')
    save_option = input().strip().lower()
    
    if save_option == 'y':
        with open('initial_draft.txt', 'w') as f:
            f.write(initial_draft)
        
        with open('refined_email.txt', 'w') as f:
            f.write(refined_email)
        
        with open('variant_email.txt', 'w') as f:
            f.write(variant_email)
        
        print("Emails have been saved to initial_draft.txt, refined_email.txt, and variant_email.txt")
    
    return initial_draft, refined_email, variant_email

if __name__ == "__main__":
    initial, refined, variant = create_email_assistant()


What's the purpose of your email?
Example: 'I need to ask my manager for time off next week due to a family emergency.'
Your description: 
=== Initial Draft ===
Subject: Task Handover for Collaboration and Review

Hi John,

I hope this message finds you well. I am writing to request that you hand over the current task to me so I can add my contributions to it. Once I have incorporated my part, I will send it to Alice for her review. After she has reviewed it, it will be referred back to me for any final adjustments.

Please let me know if you need any additional information or if there are specific files or details I need to be aware of. I appreciate your cooperation and look forward to collaborating effectively.

Thank you.

Best regards,

[Your Name]
[Your Position]
[Your Contact Information]

=== Refined Email ===
Subject: Request for Task Handover and Coordination

Dear John,

I hope this email finds you well. I am reaching out to kindly request that you transfer the current task 

##  Quasi-Agent: Email Writing Assistant

- Takes user input about their email purpose
- Generates an initial email draft (Step 1)
- Refines the tone and grammar (Step 2)
- Creates variants or follow-up emails based on user choice (Step 3)
- Optionally saves all versions to text files

In [ ]:
from litellm import completion
from typing import List, Dict
import sys

def generate_response(messages: List[Dict]) ->str:
    """Call LLm to generate response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1000
    )
return response.choices[0].message.content

def extract_code_block(response: str) ->str:
    """Extract Code block from response"""
    if '```' not in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith("python"):
        code_block=code_block[6:]
        return code_block
    def develope_custom_function():
        print("\nWhat's your kind of function you want to create?")
        print("Your Description: ", end='')    

        function_description = input().strip() #Asks the user for input and clean it up

        messages = [
            {"role": "system", "content": "You are a python expert helping to develope a function."}
        ]

        messages.append({
            "role": "user", "content": f"Write a python function that {function_description}. output the function in a ```python code block```."
            })

        initial_function = generate_response(messages)
        initial_function = extract_code_block(initial_function)
        print("\n===initial function: ===")
        print(initial_function)

    